In [1]:
from arbok.core.sequence import Sequence
from arbok.core.program import Program
from arbok.core.sample import Sample

2023-09-14 15:17:43,760 - qm - INFO     - Starting session: ab772e90-b08e-4b3e-8339-5763618af9e5


In [2]:
from arbok.samples.sunshine.configs.rf2v_config import rf2v_config
from arbok.samples.sunshine.initialization.mixed_down_up_init import (
    MixedDownUpInit
)
from arbok.samples.sunshine.readout.up_down_read import UpDownReadout

In [3]:
from arbok.samples.aurora.configs.aurora_qua_config import aurora_qua_config

from arbok.samples.aurora.initialization.odd_6_dot_init import InitOdd6dot
from arbok.samples.aurora.control.rabi_calibration import RabiCalibration
from arbok.samples.aurora.readout.psb_6_dot_read import PSB6dotRead

In [4]:
sunshine = Sample('sunshine', rf2v_config)
MEAS = Program('MEAS', sample = sunshine)

In [5]:
aurora = Sample('aurora', aurora_qua_config)
MEASA = Program('MEASA', sample = aurora)

In [6]:
MEASA.connect_OPX(host_ip= '192.168.0.3')

2023-09-14 15:17:43,933 - qm - INFO     - Performing health check
2023-09-14 15:17:43,971 - qm - INFO     - Health check passed
2023-09-14 15:17:46,005 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing existing identical Quantum Machine qm-1694669178318
2023-09-14 15:17:46,005 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1694669178318 is canceling running job 1684735462106.


In [7]:
init = MixedDownUpInit('init')
read = UpDownReadout('read')
init6 = InitOdd6dot('init6')
rabi6 = RabiCalibration('rabi6')
read6 = PSB6dotRead('read6')

In [8]:
# init6.tInitPreLoadMixed(100)
# init6.tInitPreLoadRampMixed(100)
# read6.tHomeWait(100)
# read6.tPreRead(100)

In [9]:
MEAS.add_subsequence(read)
MEASA.add_subsequence(init6)
MEASA.add_subsequence(rabi6)
MEASA.add_subsequence(read6)

In [10]:
#simulated_job = MEASA.run_local_simulation(120000)

# Running the program in the quantify framework

In [11]:
import numpy as np

In [12]:
import quantify_core.data.handling as dh
from quantify_core.measurement import MeasurementControl
from quantify_core.visualization.pyqt_plotmon import PlotMonitor_pyqt

In [13]:
#dh.set_datadir(dh.default_datadir())
dh.set_datadir()

Data will be saved in:
C:\Users\labuser\quantify-data


In [14]:
meas_ctrl = MeasurementControl('meas_ctrl')

In [15]:
plotmon = PlotMonitor_pyqt("plotmon")
meas_ctrl.instr_plotmon(plotmon.name)

C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots is deprecated. Use "qcodes_loop.plots" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots.colors is deprecated. Use "qcodes_loop.plots.colors" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\qcodes\plots\pyqtgraph.py:17: QCoDeSDeprecationWarning: The qcodes.utils.qt_helpers module is deprecated. Use "qcodes_loop.utils.qt_helpers" as an alternative.
  import qcodes.utils.qt_helpers
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:19: QCoDeSDeprecationWarning: The qcodes.plots.pyqtgraph is deprecated. Use "qcodes_loop.plots

In [16]:
f_detune1=np.arange(32.5e6,35.5e6,0.05e6).astype(int)
f_detune2=np.arange(69.5e6,72.5e6,0.05e6).astype(int)
rabi_amp_range = np.arange(0, 1, 0.04)
print(np.shape(rabi_amp_range))
rabi_freq_range = np.arange(-20e6,20e6,1e6, dtype = int)
print(np.shape(rabi_freq_range))
rabi_time_range = np.arange(4, 0.5e3+4, 0.025e3, dtype = int)
print( np.shape(rabi_time_range))

(25,)
(40,)
(20,)


In [17]:
MEASA.settables = [
    MEASA.rabi6.tRabi,
    MEASA.rabi6.IfQ1]
MEASA.setpoints_grid = [
    rabi_time_range,
    f_detune1]
MEASA.gettables = [
    MEASA.read6.diff_set1_state,
    MEASA.read6.diff_set2_state]

In [19]:
MEASA.prepare_meas_ctrl(meas_ctrl)

In [20]:
MEASA.read6.diff_set1_state.batch_size

1200

In [21]:
MEASA.read6.diff_set2_state.can_resume

True

In [22]:
# meas_ctrl.settables([
#     MEASA.rabi6.tRabi,
#     MEASA.rabi6.IfQ1])
# meas_ctrl.setpoints_grid([
#     rabi_time_range,
#     rabi_freq_range])
# meas_ctrl.gettables([
#     MEASA.read6.diff_set1_state,
#     MEASA.read6.diff_set2_state])

In [23]:
MEASA.run()

Adding qua <class 'int'> variable for tRabi
Adding qua <class 'int'> variable for IfQ1
Adding qua sweep loop for IfQ1
Adding qua sweep loop for tRabi
2023-09-14 15:18:05,960 - qm - INFO     - Sending program to QOP for compilation
2023-09-14 15:18:25,466 - qm - INFO     - Executing program


In [27]:
meas_ctrl.run("avg3", soft_avg=20)

Starting batched measurement...
Iterative settable(s) [outer loop(s)]:
	 --- (None) --- 
Batched settable(s):
	 tRabi, IfQ1 
Batch size limit: 1200

FETC 1200 1200 diff_set1_state
READ 1200 1200
2023-09-14 15:18:31,756 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1200,)
FETC 1200 1200 diff_set2_state
READ 1200 1200
2023-09-14 15:18:32,487 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1200,)
  5% completed | elapsed time:      1s | time left:     31s  last batch size:   1200  FETC 1486 2400 diff_set1_state
WAIT 1486 2400
WAIT 1786 2400
WAIT 2087 2400
WAIT 2383 2400
READ 2400 2400
2023-09-14 15:18:34,348 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1200,)
FETC 2400 2400 diff_set2_state
READ 2400 2400
2023-09-14 15:18:34,840 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1200,)
 10% completed | elapsed time:      3s | 

<xarray.Dataset>
Dimensions:  (dim_0: 1200)
Coordinates:
    x0       (dim_0) int32 4 29 54 79 104 129 154 ... 354 379 404 429 454 479
    x1       (dim_0) int32 32500000 32500000 32500000 ... 35450000 35450000
Dimensions without coordinates: dim_0
Data variables:
    y0       (dim_0) float64 0.95 1.0 0.9 0.95 1.0 1.0 ... 1.0 0.9 1.0 0.95 1.0
    y1       (dim_0) float64 0.05 0.0 0.05 0.0 0.05 ... 0.05 0.05 0.0 0.0 0.05
Attributes:
    tuid:                             20230914-151831-278-7e3354
    name:                             avg3
    grid_2d:                          True
    grid_2d_uniformly_spaced:         True
    1d_2_settables_uniformly_spaced:  False
    xlen:                             20
    ylen:                             60

In [28]:
from itertools import product

import numpy as np

import qcodes as qc
from qcodes.dataset import LinSweep, Measurement, plot_dataset
from qcodes.parameters import Parameter, ParameterWithSetpoints
from qcodes.validators import Arrays

In [29]:
def get_setpoints_array():
    return np.linspace(-5, 5, 11)


def get_pws_results():
    setpoints_arr = get_setpoints_array()
    return setpoints_arr**2 * set1()


setpoint_array = Parameter("setpoints",
                            get_cmd = get_setpoints_array,
                            vals=Arrays(shape=(11,))
                            )

pws = ParameterWithSetpoints("pws",
                             setpoints = (setpoint_array,),
                             get_cmd = get_pws_results,
                             vals=Arrays(shape=(11,))
                             )